---
---
# **Greeks**
---
---
#### ***Delta, Gamma, Theta, Vega, and Rho***
- calculations to measure different factors that might affect the price of an options contract 

### Import *.py* Files

In [15]:
# only shows data first time - restart kernel 
import CONFIG as c
import BLACK_SCHOLES as bs
import GREEKS as g

#### Import Libraries 

In [16]:
import pandas as pd
import numpy as np 

from datetime import datetime,date
from scipy.stats import norm
from math import log, sqrt, pi, exp

# Delta
---
- sensitivity of an option's price changes relative to the changes in underlying asset's prices 
- snapshot in time
- gauge likelihood option will expire *in-the-money* (ITM)
    - strik price below (for calls) or above (for puts) the underlying security's market price (MP)
- How much an Option's price can be expected to move for every `$1` change in the price of the underlying security or index 
    - Delta of `0.40` -> price will theoretically move `$0.40` for every `$1` change in the price of the underlying security or index 
    - Higher the Delta -> Bigger the Price Change 
- Number of shares of the underlying stock the option behaves like
    - Delta of `0.40` -> given a `$1` move in the underlying stock, would be comparable to owning `40` shares of the stock 

> Call Options:
>> - Positive range from `0:1`
>> - *at-the-money* -> options delta near `0.50`
>> - *in-the-money* -> get closer to `1` as expiration approaches 
>> - *out-of-the-money* -> closer to `0` as expiration approaches 

> Put Options: 
>> - negative range from `0:-1`
>> - *at-the-money* -> options delta near `-0.50`
>> - *in-the-money* -> closer to `-1` as expiration approaches
>> - *out-of-the-money* -> closer to `0` as expiration approaches 

In [17]:
# From .py File 
print('Call Delta: {}'.format(g.DELTA.call(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)))
print('Put Delta: {}'.format(g.DELTA.put(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)))

Call Delta: 0.8687937432584167
Put Delta: -0.13120625674158332


In [18]:
# Call Option: Delta
def call_delta(s,k,t,r,sigma): 
    return norm.cdf(bs.d1(s,k,t,r,sigma))

# Put Option: Delta
def put_delta(s,k,t,r,sigma): 
    return -norm.cdf(-bs.d1(s,k,t,r,sigma))

In [19]:
cDelta = call_delta(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)
pDelta = put_delta(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)

print('Call Option Delta: {}'.format(cDelta),end='\n')
print('Put Option Delta: {}'.format(pDelta),end='\n')

Call Option Delta: 0.8687937432584167
Put Option Delta: -0.13120625674158332


# Gamma
---
- measures rate of change in an option's *Delta* over time 
    - per `$1` change in the price of the underlying security/index
- *Delta* went from `0.40` to `0.55` -> Change in *Delta* of `0.15` is the option's **Gamma**
- **Gamma** decreases as an option becomes more ITM and *Delta* approaches `1`

In [20]:
# From .py File
print('Call Gamma: {}'.format(g.GAMMA.call(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)))
print('Put Gamma: {}'.format(g.GAMMA.put(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)))

Call Gamma: 0.003953327689004186
Put Gamma: 0.003953327689004186


In [21]:
# Call Option: Gamma
def call_gamma(s,k,t,r,sigma): 
    return norm.pdf(bs.d1(s,k,t,r,sigma)) / (s*sigma*sqrt(t))

# Put Option: Gamma
def put_gamma(s,k,t,r,sigma): 
    return norm.pdf(bs.d1(s,k,t,r,sigma)) / (s*sigma*sqrt(t))

In [22]:
cGamma = call_gamma(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)
pGamma = put_gamma(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)

print('Call Option Gamma: {}'.format(cGamma),end='\n')
print('Put Option Gamma: {}'.format(pGamma),end='\n')

Call Option Gamma: 0.003953327689004186
Put Option Gamma: 0.003953327689004186


# Vega
---
- sensitivity of an option price relative to the volatility of the underlying asset 
- measures rate of change in an option's price per *one-percentage-point-change* in the **implied volatility** of the underlying stock 
- How much an option's price SHOULD MOVE when the volatility of the underlying security/index increases or decreases 
- Drop in Vega: cause both calls and puts to lose value
- Increase in Vega: cause both calls and puts to gain value 
- Ignoring Vega can cause you to potentially overpay when buying otpions
    - consider buying options when vega is below 'normal' levels 
    - consider selling options when vega is above 'normal' levels 
    - find 'normal' by comparing historical volatility to the implied volatility 

In [23]:
# From .py File
print('Call Vega: {}'.format(g.VEGA.call(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)))
print('Put Vega: {}'.format(g.VEGA.put(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)))

AttributeError: type object 'VEGA' has no attribute 'call'

In [ ]:
# Call Option: Vega
def call_vega(s,k,t,r,sigma): 
    return 0.01 * (s * norm.pdf(bs.d1(s,k,t,r,sigma))*sqrt(t))

# Put Option: Gamma
def put_vega(s,k,t,r,sigma): 
    return 0.01 * (s * norm.pdf(bs.d1(s,k,t,r,sigma))*sqrt(t))

In [ ]:
cVega = call_vega(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)
pVega = put_vega(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)

print('Call Option Vega: {}'.format(cVega),end='\n')
print('Put Option Vega: {}'.format(pVega),end='\n')

# Theta
---
- sensitivity of option price relative to the option's time to maturity 
- how much the price of an option should decrease each day as the option nears expiration 
- price erosion over time: **time decay**
- Time-Value erosion is not linear
    - ATM just slightly OOTM as expiration approaches
    - ITM increases as expiration approaches 
    - OOTM decrease as expiration approaches

In [24]:
# From .py file
print('Call Theta: {}'.format(g.THETA.call(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)))
print('Put Theta: {}'.format(g.THETA.put(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)))

AttributeError: type object 'THETA' has no attribute 'call'

In [ ]:
# Call Option: Theta
def call_theta(s,k,t,r,sigma): 
    return 0.01 * (-(s * norm.pdf(bs.d1(s,k,t,r,sigma))*sigma)/(2*sqrt(t)) - r*k*exp(-r*t)*norm.cdf(bs.d2(s,k,t,r,sigma)))

# Put Option: Theta
def put_theta(s,k,t,r,sigma): 
    return 0.01 * (-(s * norm.pdf(bs.d1(s,k,t,r,sigma))*sigma)/(2*sqrt(t)) + r*k*exp(-r*t)*norm.cdf(-bs.d2(s,k,t,r,sigma)))

In [ ]:
cTheta = call_theta(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)
pTheta = put_theta(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)

print('Call Option Theta: {}'.format(cTheta),end='\n')
print('Put Option Theta: {}'.format(pTheta),end='\n')

---
## Rho
---
- sensitivity of the option price relative to interest rates
- measures expected change in an option's price per *one-percentage-point* change in interest rates 
    - how much the price of an option should rise or fall if the risk-free interest rate (US T-Bills) increases or decreases
    - impliest potential change in interest rates 
    - applied around FOMC meetings 
- IR Increase: value of **call options** increase
    - Call Options: Positive Rho
- IR Decrease: value of **put options** decrease 
    - Put Options: Negative Rho 
- stock trading at strike price of `$25`
    - 25 calls and 25 puts -> exactly at the money
    - calls trading at `$0.60`
    - puts trading at `$0.50`
    - when IR are low, price difference between puts and calls will be relatively small
    - when IR increase, gap widens - calls will become more expensive 
- Long-Term Equity AnticiPation Securities (LEAPS) far more sensitive to change in IR than short-term options 
  

In [26]:
# From .py File
print('Call Rho: {}'.format(g.RHO.call(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)))
print('Put Rho: {}'.format(g.RHO.put(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)))

AttributeError: type object 'RHO' has no attribute 'call'

In [ ]:
# Call Option: Rho
def call_rho(s,k,t,r,sigma): 
    return 0.01 * (k*t*exp(-r*t) * norm.cdf(bs.d2(s,k,t,r,sigma)))

# Put Option: Rho
def put_rho(s,k,t,r,sigma): 
    return 0.01 * (-k*t*exp(-r*t) * norm.cdf(-bs.d2(s,k,t,r,sigma)))

In [ ]:
cRho = call_rho(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)
pRho = put_rho(bs.lastCloseP, c.strike_price, bs.tMature, bs.riskFree, bs.sigma)

print('Call Option Rho: {}'.format(cRho),end='\n')
print('Put Option Rho: {}'.format(pRho),end='\n')